<a href="https://colab.research.google.com/github/harish-git63/Demo1/blob/main/HARISH_STUDENT_DATABASE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import sqlite3
import tkinter as tk
from tkinter import messagebox, ttk

# Connect to SQLite database
conn = sqlite3.connect('students.db')
cursor = conn.cursor()

# Create table if it doesn't exist
cursor.execute('''
    CREATE TABLE IF NOT EXISTS students (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        register_number TEXT UNIQUE NOT NULL,
        department TEXT NOT NULL
    )
''')
conn.commit()

def clear_entries():
    name_entry.delete(0, tk.END)
    reg_entry.delete(0, tk.END)
    dept_entry.delete(0, tk.END)

# Functions for database operations
def add_student():
    name = name_entry.get().strip()
    reg_no = reg_entry.get().strip()
    dept = dept_entry.get().strip()
    print("[DEBUG] Adding:", name, reg_no, dept)
    try:
        cursor.execute(
            "INSERT INTO students (name, register_number, department) VALUES (?, ?, ?)", (name, reg_no, dept))
        conn.commit()
        messagebox.showinfo("Success", "Student added successfully!")
        view_students()
        clear_entries()
    except sqlite3.IntegrityError:
        messagebox.showerror("Error", "Register number already exists.")

def view_students():
    for row in tree.get_children():
        tree.delete(row)
    cursor.execute("SELECT * FROM students")
    for row in cursor.fetchall():
        tree.insert("", "end", values=row)

def update_student():
    reg_no = reg_entry.get().strip()
    new_name = name_entry.get().strip()
    new_dept = dept_entry.get().strip()
    print("[DEBUG] Updating:", reg_no, new_name, new_dept)
    cursor.execute("UPDATE students SET name = ?, department = ? WHERE register_number = ?",
                   (new_name, new_dept, reg_no))
    if cursor.rowcount == 0:
        messagebox.showerror("Error", "Student not found.")
    else:
        conn.commit()
        messagebox.showinfo("Success", "Student updated successfully!")
        view_students()
        clear_entries()

def delete_student():
    reg_no = reg_entry.get().strip()
    print("[DEBUG] Deleting:", reg_no)
    cursor.execute("DELETE FROM students WHERE register_number = ?", (reg_no,))
    if cursor.rowcount == 0:
        messagebox.showerror("Error", "Student not found.")
    else:
        conn.commit()
        messagebox.showinfo("Deleted", "Student deleted successfully!")
        view_students()
        clear_entries()

# Treeview row selection fill
def on_row_select(event):
    selected = tree.focus()
    if not selected:
        return
    values = tree.item(selected, 'values')
    if values:
        id_val, name_val, reg_val, dept_val = values
        name_entry.delete(0, tk.END)
        name_entry.insert(0, name_val)
        reg_entry.delete(0, tk.END)
        reg_entry.insert(0, reg_val)
        dept_entry.delete(0, tk.END)
        dept_entry.insert(0, dept_val)

# GUI setup
root = tk.Tk()
root.title("Student Management System")
root.geometry("700x500")

# Input fields
tk.Label(root, text="Name").grid(row=0, column=0, padx=10, pady=5, sticky='w')
name_entry = tk.Entry(root)
name_entry.grid(row=0, column=1, padx=10, pady=5)

tk.Label(root, text="Register Number").grid(row=1, column=0, padx=10, pady=5, sticky='w')
reg_entry = tk.Entry(root)
reg_entry.grid(row=1, column=1, padx=10, pady=5)

tk.Label(root, text="Department").grid(row=2, column=0, padx=10, pady=5, sticky='w')
dept_entry = tk.Entry(root)
dept_entry.grid(row=2, column=1, padx=10, pady=5)

# Buttons
tk.Button(root, text="Add Student", command=add_student, bg="lightgreen").grid(row=3, column=0, padx=10, pady=10)
tk.Button(root, text="View Students", command=view_students, bg="lightblue").grid(row=3, column=1, padx=10, pady=10)
tk.Button(root, text="Update Student", command=update_student, bg="khaki").grid(row=4, column=0, padx=10, pady=10)
tk.Button(root, text="Delete Student", command=delete_student, bg="salmon").grid(row=4, column=1, padx=10, pady=10)

# Treeview for displaying records
columns = ("ID", "Name", "Register Number", "Department")
tree = ttk.Treeview(root, columns=columns, show="headings")
for col in columns:
    tree.heading(col, text=col)
    tree.column(col, width=150)
tree.grid(row=5, column=0, columnspan=2, padx=10, pady=20)
tree.bind("<<TreeviewSelect>>", on_row_select)

view_students()
root.mainloop()
conn.close()


TclError: no display name and no $DISPLAY environment variable